In [ ]:
! pip install openpyxl  vectorbt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 527.3/527.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 23.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.5/3.5 MB 52.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.6/34.6 MB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 37.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 56.3 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 1.25.2
    Uninstalling numpy-1.25.2:
      Successfully uninstalled numpy-1.25.2
  Attempting uninstall: llvmlite
    Found existing installation: llvmlite 0.41.1
    Uninstalling llvmlite-0.41.1:
      Successfully uninstalled llvmlite-0.41.1
  Attempting uninstall: numba
    Found existing installation: numba 0.58.1
    Uninstalling numba-0.58.1:
      Succes

In [ ]:
import pandas as pd
import numpy as np
#import ta
import vectorbt as vbt
import plotly.graph_objects as go

In [ ]:
import os
import glob

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!unzip -q "/content/drive/MyDrive/Commercial_Bank.zip" -d "/content"

In [ ]:
import os
import pandas as pd

directory_path = '/content/Commercial_Bank/NMB_Bank_Limited/Price_History'
csv_files = sorted([os.path.join(directory_path, file) for file in os.listdir(directory_path) if file.endswith('.csv')])
dataframes = []

for file in csv_files:
    df = pd.read_csv(file)
    df.rename(columns={'Ltp': 'Close', 'date': 'Date', 'open': 'Open', 'high': 'High', 'low': 'Low', 'volume': 'Qty'}, inplace=True)
    df.set_index('Date', inplace=True)
    dataframes.append(df)

NMB = pd.concat(dataframes)

# Sort the DataFrame by 'Date' in descending order
NMB.sort_index(ascending=False, inplace=True)

# Display the DataFrame
print(NMB)


            S.N.   Open   High    Low  Close  % Change        Qty  \
Date                                                                
2024-04-15     2  175.0  177.0  173.2  176.0      0.00  28,531.00   
2024-04-14     3  179.0  179.0  175.0  176.0     -0.62  33,351.00   
2024-04-10     4  177.0  179.0  174.0  177.1     -0.06  37,688.00   
2024-04-09     5  177.0  178.5  175.7  177.2      0.74  29,320.00   
2024-04-07     6  177.0  177.0  172.5  175.9     -0.62  72,850.00   
...          ...    ...    ...    ...    ...       ...        ...   
2011-03-28  2603  245.0  244.0  226.0  228.0      0.00   1,951.00   
2011-03-27  2604  249.0  245.0  245.0  245.0      0.00     636.00   
2011-03-25  2605  254.0  249.0  249.0  249.0      0.00      88.00   
2011-03-24  2606  260.0  255.0  254.0  254.0      0.00     344.00   
2011-03-20  2607  251.0  251.0  247.0  250.0      0.00     599.00   

                 Turnover  
Date                       
2024-04-15   4,994,498.40  
2024-04-14   5,889

In [ ]:
NMB.head()

,S.N.,Open,High,Low,Close,% Change,Qty,Turnover
Date,,,,,,,,
2024-04-15,2,175.0,177.0,173.2,176.0,0.00,"28,531.00","4,994,498.40"
2024-04-14,3,179.0,179.0,175.0,176.0,-0.62,"33,351.00","5,889,814.70"
2024-04-10,4,177.0,179.0,174.0,177.1,-0.06,"37,688.00","6,671,737.40"
2024-04-09,5,177.0,178.5,175.7,177.2,0.74,"29,320.00","5,189,715.80"
2024-04-07,6,177.0,177.0,172.5,175.9,-0.62,"72,850.00","12,711,339.90"


**MACD**

In [ ]:
import plotly.graph_objects as go

def MACD(df, slow_window=26, fast_window=12, signal_period=9):

    '''
    Calculates the Moving Average Convergence Divergence (MACD) lines and signal line,
    generates entry and exit signals based on MACD crossovers, determines the current market trend,
    and visualizes the market scenario with MACD lines and signals.

    Parameters:
    - df (pd.DataFrame): DataFrame containing financial data with a 'Close' column.
    - slow_window (int): Window size for the slower moving average (default is 26).
    - fast_window (int): Window size for the faster moving average (default is 12).
    - signal_period (int): Window size for the signal line (default is 9).

    Returns:
    - pf (vectorbt.portfolio.base.Portfolio): Portfolio object containing trade signals.
    - current_entry (tuple): Tuple containing index and price of the most recent entry point.
    - current_trend (str): Description of the current market trend (Uptrend, Downtrend, or Sideways).
    '''

    #df['High'] = df['High'].astype(str).str.replace(',', '').astype(float)
    #df['Low'] = df['Low'].astype(str).str.replace(',', '').astype(float)
    df['Close'] = df['Close'].astype(str).str.replace(',', '').astype(float)
    #df['Qty'] = df['Qty'].astype(str).str.replace(',', '').astype(float)

    close = df['Close']

    # Calculate MACD lines and signal line
    macd = vbt.MACD.run(close, slow_window=slow_window, fast_window=fast_window, signal_window=signal_period)

    # Generate entry and exit signals based on MACD crossovers
    entries = (macd.macd > macd.signal)
    exits = (macd.macd < macd.signal)

    # Determine the current market trend based on recent MACD crossovers
    if entries.iloc[-1]:
        current_trend = "Uptrend"
    elif exits.iloc[-1]:
        current_trend = "Downtrend"
    else:
        current_trend = "Sideways"

    # Create a portfolio from signals
    pf = vbt.Portfolio.from_signals(close, entries, exits)

    # Find the most recent entry point
    if entries.any():
        last_entry_index = entries[entries].index[1]
        current_entry = (last_entry_index, close[last_entry_index])
    else:
        current_entry = (None, None)

    # Plotting
    fig = go.Figure()

    # Plot the closing price
    fig.add_trace(go.Scatter(x=close.index, y=close.values, mode='lines', name='Close'))

    # Plot the MACD lines and signal line
    fig.add_trace(go.Scatter(x=macd.macd.index, y=macd.macd.values, mode='lines', name='MACD Line'))
    fig.add_trace(go.Scatter(x=macd.signal.index, y=macd.signal.values, mode='lines', name='Signal Line'))

    # Plot entry signals
    fig.add_trace(go.Scatter(
        x=entries[entries].index,
        y=close[entries],
        mode='markers',
        marker=dict(symbol='triangle-up', color='green', size=10),
        name='Entry Signal'
    ))
#
    ## Plot exit signals
    fig.add_trace(go.Scatter(
        x=exits[exits].index,
        y=close[exits],
        mode='markers',
        marker=dict(symbol='triangle-down', color='red', size=10),
        name='Exit Signal'
    ))
#
    ## Add titles and labels
    fig.update_layout(
        title='Market Scenario with MACD',
        xaxis_title='Date',
        yaxis_title='Price',
        legend_title='Legend'
    )
#
    ## Show the plot
    fig.show()
    return pf, current_entry, current_trend

In [ ]:
pf, current_entry, current_trend = MACD(NMB, fast_window = 5, slow_window = 10)
print(f"The current market trend according to the most recent 10 moving average cross is {current_trend}")
print(f"According to the moving averages of fast_window = 5 and slow_window = 20, the backtested return is: {pf.total_return()}")
print(f"the current entry is at Date: {current_entry[0]} and value {current_entry[1]}")

The current market trend according to the most recent 10 moving average cross is Uptrend
According to the moving averages of fast_window = 5 and slow_window = 20, the backtested return is: -0.005675687119474446
the current entry is at Date: 2024-03-17 and value 194.3


**Moving Average**

In [ ]:
import plotly.graph_objects as go

def Moving_average(df, fast_window=5, slow_window=20):
    '''
    Calculates fast and slow moving averages, generates entry and exit signals based on moving average crossovers,
    determines the current trend, and visualizes the market scenario with candlestick chart and moving averages.

    Parameters:
    - df (pd.DataFrame): DataFrame containing OHLC (Open, High, Low, Close) data.
    - fast_window (int): Window size for fast moving average calculation (default is 5).
    - slow_window (int): Window size for slow moving average calculation (default is 20).

    Returns:
    - pf (vectorbt.portfolio.base.Portfolio): Portfolio object containing trade signals.
    - current_entry (tuple): Tuple containing index and price of the most recent entry point.
    - current_trend (str): Description of the current market trend (Uptrend or Downtrend).
    '''
    #df['High'] = df['High'].astype(str).str.replace(',', '').astype(float)
    #df['Low'] = df['Low'].astype(str).str.replace(',', '').astype(float)
    df['Close'] = df['Close'].astype(str).str.replace(',', '').astype(float)
    #df['Qty'] = df['Qty'].astype(str).str.replace(',', '').astype(float)

    # Calculate fast and slow moving averages
    fast_ma = vbt.MA.run(df['Close'], window=[fast_window], param_product=True)
    slow_ma = vbt.MA.run(df['Close'], window=[slow_window], param_product=True)

    # Generate entry and exit signals based on moving average crossovers
    entries = fast_ma.ma_crossed_below(slow_ma)
    exits = fast_ma.ma_crossed_above(slow_ma)

    # Determine the current trend
    trend_values = fast_ma.ma.iloc[-20:] > slow_ma.ma.iloc[-20:]  # Check last 20 periods
    trend_count = trend_values.sum()
    current_trend = "Uptrend" if trend_count > 5 else "Downtrend"  # If more than 5 out of last 20 periods show uptrend, consider as uptrend

    # Create a portfolio tracker
    pf = vbt.Portfolio.from_signals(df['Close'], entries, exits)

    # Find the most recent entry point
    if entries.any():
        last_entry_index = entries[entries].index[1]  # Assuming index 1 since index 0 may be the first entry
        current_entry = (last_entry_index, df['Close'][last_entry_index])
    else:
        current_entry = (None, None)

    # Plotting
    fig = go.Figure()

    # Plot the candlestick chart
    fig.add_trace(go.Candlestick(x=df.index,
                                 open=df['Open'],
                                 high=df['High'],
                                 low=df['Low'],
                                 close=df['Close'],
                                 name='Candlestick'))

    # Plot the fast moving average
    fig.add_trace(go.Scatter(x=df.index,
                             y=fast_ma.ma.values,
                             mode='lines',
                             name=f'Fast MA ({fast_window})'))

    # Plot the slow moving average
    fig.add_trace(go.Scatter(x=df.index,
                             y=slow_ma.ma.values,
                             mode='lines',
                             name=f'Slow MA ({slow_window})'))

    # Plot entry signals
    fig.add_trace(go.Scatter(x=entries[entries].index,
                             y=df['Close'][entries],
                             mode='markers',
                             marker=dict(symbol='triangle-up', color='green', size=10),
                             name='Entry Signal'))

    # Plot exit signals
    fig.add_trace(go.Scatter(x=exits[exits].index,
                             y=df['Close'][exits],
                             mode='markers',
                             marker=dict(symbol='triangle-down', color='red', size=10),
                             name='Exit Signal'))

    # Add titles and labels
    fig.update_layout(title='Market Scenario with Moving Averages',
                      xaxis_title='Date',
                      yaxis_title='Price',
                      legend_title='Legend')

    fig.show()
    return pf, current_entry, current_trend


In [ ]:
pf, current_entry, current_trend = Moving_average(NMB, fast_window = 10, slow_window = 50)
print(f"The current market trend according to the most recent 10 moving average cross is {current_trend}")
print(f"According to the moving averages of fast_window = 5 and slow_window = 20, the backtested return is: {pf.total_return()}")
print(f"the current entry is at Date: {current_entry[0]} and value {current_entry[1]}")

The current market trend according to the most recent 10 moving average cross is Uptrend
According to the moving averages of fast_window = 5 and slow_window = 20, the backtested return is: 0.3907481714400978
the current entry is at Date: 2023-07-05 and value 223.0


**Relative Signal Index(RSI)**

In [ ]:
import plotly.graph_objects as go
import pandas as pd
import vectorbt as vbt

def RSI(df, window=14, overbought=70, oversold=30):

    '''
    Calculates the Relative Strength Index (RSI) and generates entry and exit signals based on specified parameters.

    Parameters:
    - df (pd.DataFrame): DataFrame containing stock data with a 'Close' column.
    - window (int): Window size for RSI calculation (default is 14).
    - overbought (float): Threshold for overbought condition (default is 70).
    - oversold (float): Threshold for oversold condition (default is 30).

    Returns:
    - pf (vectorbt.portfolio.base.Portfolio): Portfolio object containing trade signals.
    - current_entry (tuple): Tuple containing index and price of the most recent entry point.
    - current_trend (str): Description of the current market trend based on recent RSI values.
    - latest_rsi (float): The latest RSI value.
    '''
    # Convert 'Close' column to float
    df['Close'] = df['Close'].astype(str).str.replace(',', '').astype(float)

    close = df['Close']

    # Calculate RSI
    rsi = vbt.RSI.run(close, window=window)

    # Generate entry and exit signals based on RSI
    entries = rsi.rsi_crossed_below(oversold)
    exits = rsi.rsi_crossed_above(overbought)

    # Determine the current market trend based on recent RSI values
    if rsi.rsi.iloc[-1] < oversold:
        current_trend = "Uptrend (Potential Reversal)"
    elif rsi.rsi.iloc[-1] > overbought:
        current_trend = "Downtrend (Potential Reversal)"
    else:
        current_trend = "Sideways"

    # Get the latest RSI value
    latest_rsi = rsi.rsi.iloc[-1]

    # Create a portfolio from signals
    pf = vbt.Portfolio.from_signals(close, entries, exits)

    # Find the most recent entry point
    if entries.any():
        last_entry_index = entries[entries].index[-1]
        current_entry = (last_entry_index, close[last_entry_index])
    else:
        current_entry = (None, None)

    # Plotting
    fig = go.Figure()

    # Plot the closing price
    fig.add_trace(go.Scatter(x=close.index, y=close.values, mode='lines', name='Close'))

    # Plot the RSI
    fig.add_trace(go.Scatter(x=rsi.rsi.index, y=rsi.rsi.values, mode='lines', name='RSI'))

    # Plot overbought and oversold lines
    fig.add_trace(go.Scatter(
        x=[close.index[0], close.index[-1]],
        y=[overbought, overbought],
        mode='lines',
        line=dict(dash='dash', color='red'),
        name='Overbought'
    ))

    fig.add_trace(go.Scatter(
        x=[close.index[0], close.index[-1]],
        y=[oversold, oversold],
        mode='lines',
        line=dict(dash='dash', color='green'),
        name='Oversold'
    ))

    # Plot entry signals
    fig.add_trace(go.Scatter(
        x=entries[entries].index,
        y=close[entries],
        mode='markers',
        marker=dict(symbol='triangle-up', color='green', size=10),
        name='Entry Signal'
    ))

    # Plot exit signals
    fig.add_trace(go.Scatter(
        x=exits[exits].index,
        y=close[exits],
        mode='markers',
        marker=dict(symbol='triangle-down', color='red', size=10),
        name='Exit Signal'
    ))

    # Add titles and labels
    fig.update_layout(
        title='Market Scenario with RSI',
        xaxis_title='Date',
        yaxis_title='Price',
        legend_title='Legend'
    )

    # Show the plot
    fig.show()

    return pf, current_entry, current_trend, latest_rsi


In [ ]:
# Run the RSI function
portfolio, last_entry, trend, RSI_Score = RSI(NMB, window=14)

# Print the results
print(f"The current market trend using RSI Analysis is {current_trend}")
print(f"According to the window size of 14, the backtested return is: {pf.total_return()}")
print(f"the current entry is at Date: {current_entry[0]} and value {current_entry[1]}")
print(f"The latest RSI value is {RSI_Score}")

The current market trend using RSI Analysis is Uptrend
According to the window size of 14, the backtested return is: 0.3907481714400978
the current entry is at Date: 2023-07-05 and value 223.0
The latest RSI value is 73.97260273972603


**Average True Range (ATR)**

In [ ]:
import plotly.graph_objects as go

def ATR(df, window=14):

    '''
    Calculates the Average True Range (ATR) and visualizes it along with the closing prices.

    Parameters:
    - high (pd.Series): Series of high prices.
    - low (pd.Series): Series of low prices.
    - close (pd.Series): Series of closing prices.
    - window (int): Window size for ATR calculation (default is 14).

    Returns:
    - atr (pd.DataFrame): DataFrame containing ATR values.
    - current_atr (float): The most recent ATR value.
    '''
    df['High'] = df['High'].astype(str).str.replace(',', '').astype(float)
    df['Low'] = df['Low'].astype(str).str.replace(',', '').astype(float)
    df['Close'] = df['Close'].astype(str).str.replace(',', '').astype(float)
    #df['Qty'] = df['Qty'].astype(str).str.replace(',', '').astype(float)

    high = df['High']
    low = df['Low']
    close = df['Close']

    # Calculate ATR
    atr = vbt.ATR.run(high, low, close, window=window)

    # Determine the current market volatility based on recent ATR value
    current_atr = atr.atr.iloc[-1]

    # Plotting
    fig = go.Figure()

    # Plot the closing price
    fig.add_trace(go.Scatter(x=close.index, y=close.values, mode='lines', name='Close'))

    # Plot the ATR
    fig.add_trace(go.Scatter(x=atr.atr.index, y=atr.atr.values, mode='lines', name='ATR'))

    # Add titles and labels
    fig.update_layout(
        title='Market Volatility with ATR',
        xaxis_title='Date',
        yaxis_title='Price / ATR',
        legend_title='Legend'
    )

    # Show the plot
    fig.show()

    return atr, current_atr


In [ ]:
# Run the ATR function
atr_data, current_volatility = ATR(NMB, window=14)

# Print the results
#print("ATR Data:", atr_data)
print("Current ATR (Volatility):", current_volatility)

Current ATR (Volatility): 8.678588145732364


**Bollinger Band**

In [ ]:
import plotly.graph_objects as go
import pandas as pd

def BollingerBands(df, window=20, num_std=2):

    '''
    Calculates Bollinger Bands and generates entry and exit signals based on specified parameters.

    Parameters:
    - dataframe (pd.DataFrame): DataFrame containing historical price data, including a 'Close' column.
    - window (int): Window size for calculating the moving average (default is 20).
    - num_std (int): Number of standard deviations to use for calculating the Bollinger Bands (default is 2).

    Returns:
    - pf (vectorbt.portfolio.base.Portfolio): Portfolio object containing trade signals.
    - current_entry (tuple): Tuple containing index and price of the most recent entry point.
    '''

    #df['High'] = df['High'].astype(str).str.replace(',', '').astype(float)
    #df['Low'] = df['Low'].astype(str).str.replace(',', '').astype(float)
    df['Close'] = df['Close'].astype(str).str.replace(',', '').astype(float)
    #df['Qty'] = df['Qty'].astype(str).str.replace(',', '').astype(float)

    close = df['Close']

    # Calculate the rolling mean (SMA) and standard deviation
    sma = close.rolling(window=window).mean()
    std = close.rolling(window=window).std()

    # Calculate upper and lower Bollinger Bands
    upper_band = sma + num_std * std
    lower_band = sma - num_std * std

    # Generate entry and exit signals based on Bollinger Bands
    entries = close < lower_band
    exits = close > upper_band

    # Create a portfolio from signals
    pf = vbt.Portfolio.from_signals(close, entries, exits)

    # Find the most recent entry point
    if entries.any():
        last_entry_index = entries[entries].index[-1]
        current_entry = (last_entry_index, close[last_entry_index])
    else:
        current_entry = (None, None)

    # Plotting
    fig = go.Figure()

    # Plot the closing price
    #fig.add_trace(go.Scatter(x=close.index, y=close.values, mode='lines', name='Close'))

    # Plot the Bollinger Bands
    fig.add_trace(go.Scatter(x=sma.index, y=sma.values, mode='lines', name='SMA'))
    fig.add_trace(go.Scatter(x=upper_band.index, y=upper_band.values, mode='lines', name='Upper Band'))
    fig.add_trace(go.Scatter(x=lower_band.index, y=lower_band.values, mode='lines', name='Lower Band'))

    # Plot entry signals
    fig.add_trace(go.Scatter(
        x=entries[entries].index,
        y=close[entries],
        mode='markers',
        marker=dict(symbol='triangle-up', color='green', size=10),
        name='Entry Signal'
    ))

    # Plot exit signals
    fig.add_trace(go.Scatter(
        x=exits[exits].index,
        y=close[exits],
        mode='markers',
        marker=dict(symbol='triangle-down', color='red', size=10),
        name='Exit Signal'
    ))

    # Add titles and labels
    fig.update_layout(
        title='Bollinger Bands Strategy',
        xaxis_title='Date',
        yaxis_title='Price',
        legend_title='Legend'
    )

    # Show the plot
    fig.show()

    return pf, current_entry

In [ ]:
# Run the Bollinger Bands function
portfolio, last_entry = BollingerBands(NMB)

# Print the results
print(f"According to the window size of 20 and 2 standard deviations, the backtested return is: {pf.total_return()}")
print(f"the current entry is at Date: {current_entry[0]} and value {current_entry[1]}")

According to the window size of 20 and 2 standard deviations, the backtested return is: 0.3907481714400978
the current entry is at Date: 2023-07-05 and value 223.0


**On Balance Volume**

In [ ]:
def OBVStrategy(df, window=20):
    '''
    Calculates the On Balance Volume (OBV) and generates entry and exit signals based on OBV crossovers with its moving average.
    Determines the current market trend and visualizes the market scenario with OBV and signals.

    Parameters:
    - df (pd.DataFrame): DataFrame containing financial data with 'Close' and 'Volume' columns.
    - window (int): Window size for the moving average of OBV (default is 20).

    Returns:
    - pf (vectorbt.portfolio.base.Portfolio): Portfolio object containing trade signals.
    - current_entry (tuple): Tuple containing index and price of the most recent entry point.
    - current_trend (str): Description of the current market trend (Uptrend, Downtrend, or Sideways).
    '''
    #df['High'] = df['High'].astype(str).str.replace(',', '').astype(float)
    #df['Low'] = df['Low'].astype(str).str.replace(',', '').astype(float)
    df['Close'] = df['Close'].astype(str).str.replace(',', '').astype(float)
    df['Qty'] = df['Qty'].astype(str).str.replace(',', '').astype(float)

    close = df['Close']
    volume = df['Qty']

    # Calculate OBV
    obv = vbt.OBV.run(close, volume)

    # Calculate moving average of OBV
    obv_ma = obv.obv.rolling(window=window).mean()

    # Generate entry and exit signals based on OBV crossovers with its moving average
    entries = obv.obv_crossed_above(obv_ma)
    exits = obv.obv_crossed_below(obv_ma)

    # Determine the current market trend based on recent OBV crossovers
    if entries.iloc[-1]:
        current_trend = "Uptrend"
    elif exits.iloc[-1]:
        current_trend = "Downtrend"
    else:
        current_trend = "Sideways"

    # Create a portfolio from signals
    pf = vbt.Portfolio.from_signals(close, entries, exits)

    # Find the most recent entry point
    if entries.any():
        last_entry_index = entries[entries].index[-1]
        current_entry = (last_entry_index, close[last_entry_index])
    else:
        current_entry = (None, None)

    # Plotting
    fig = go.Figure()

    # Plot the closing price
    fig.add_trace(go.Scatter(x=close.index, y=close.values, mode='lines', name='Close'))

    # Plot the OBV
    fig.add_trace(go.Scatter(x=obv.obv.index, y=obv.obv.values, mode='lines', name='OBV'))

    # Plot the OBV moving average
    fig.add_trace(go.Scatter(x=obv_ma.index, y=obv_ma.values, mode='lines', name=f'OBV MA ({window})'))

    # Plot entry signals
    fig.add_trace(go.Scatter(
        x=entries[entries].index,
        y=close[entries],
        mode='markers',
        marker=dict(symbol='triangle-up', color='green', size=10),
        name='Entry Signal'
    ))

    # Plot exit signals
    fig.add_trace(go.Scatter(
        x=exits[exits].index,
        y=close[exits],
        mode='markers',
        marker=dict(symbol='triangle-down', color='red', size=10),
        name='Exit Signal'
    ))

    # Add titles and labels
    fig.update_layout(
        title='Market Scenario with OBV',
        xaxis_title='Date',
        yaxis_title='Volume / OBV',
        legend_title='Legend'
    )

    # Show the plot
    fig.show()
    return pf, current_entry, current_trend

In [ ]:
# Run the OBV strategy
pf, current_entry, current_trend = OBVStrategy(df)
#print the results
print(f"According to the window size of 20, the backtested return is: {pf.total_return()}")
print(f"the current entry is at Date: {current_entry[0]} and value {current_entry[1]}")

According to the window size of 20, the backtested return is: 0.08273181565796776
the current entry is at Date: 2024-02-22 and value 180.1


**Moving Standard Deviation**

In [ ]:
import vectorbt as vbt
import pandas as pd
import plotly.graph_objects as go

def MSTDStrategy(df, window=20, threshold=1.6):
    '''
    Calculates the Moving Standard Deviation (MSTD), generates entry and exit signals based on MSTD crossovers,
    determines the current market trend, and visualizes the market scenario with MSTD and signals.

    Parameters:
    - df (pd.DataFrame): DataFrame containing 'Close' prices.
    - window (int): Window size for calculating the moving standard deviation (default is 20).
    - threshold (float): Threshold for generating entry and exit signals (default is 1.5).

    Returns:
    - pf (vectorbt.portfolio.base.Portfolio): Portfolio object containing trade signals.
    - current_entry (tuple): Tuple containing index and price of the most recent entry point.
    - current_trend (str): Description of the current market trend (Uptrend, Downtrend, or Sideways).
    '''

    #df['High'] = df['High'].astype(str).str.replace(',', '').astype(float)
    #df['Low'] = df['Low'].astype(str).str.replace(',', '').astype(float)
    df['Close'] = df['Close'].astype(str).str.replace(',', '').astype(float)
    #df['Qty'] = df['Qty'].astype(str).str.replace(',', '').astype(float)

    close = df['Close']

    # Calculate the moving standard deviation (MSTD)
    mstd = close.rolling(window=window).std()

    # Generate entry and exit signals based on MSTD crossovers
    entries = close < (close.mean() - threshold * mstd)
    exits = close > (close.mean() + threshold * mstd)

    # Determine the current market trend based on recent MSTD crossovers
    if entries.iloc[-1]:
        current_trend = "Uptrend"
    elif exits.iloc[-1]:
        current_trend = "Downtrend"
    else:
        current_trend = "Sideways"

    # Create a portfolio from signals
    pf = vbt.Portfolio.from_signals(close, entries, exits)

    # Find the most recent entry point
    if entries.any():
        last_entry_index = entries[entries].index[-1]
        current_entry = (last_entry_index, close[last_entry_index])
    else:
        current_entry = (None, None)

    # Plotting
    fig = go.Figure()

    # Plot the closing price
    fig.add_trace(go.Scatter(x=close.index, y=close.values, mode='lines', name='Close'))

    # Plot the MSTD
    fig.add_trace(go.Scatter(x=mstd.index, y=mstd.values, mode='lines', name='MSTD'))

    # Plot entry signals
    fig.add_trace(go.Scatter(
        x=entries[entries].index,
        y=close[entries],
        mode='markers',
        marker=dict(symbol='triangle-up', color='green', size=10),
        name='Entry Signal'
    ))

    # Plot exit signals
    fig.add_trace(go.Scatter(
        x=exits[exits].index,
        y=close[exits],
        mode='markers',
        marker=dict(symbol='triangle-down', color='red', size=10),
        name='Exit Signal'
    ))

    # Add titles and labels
    fig.update_layout(
        title='Market Scenario with MSTD',
        xaxis_title='Date',
        yaxis_title='Price / MSTD',
        legend_title='Legend'
    )

    # Show the plot
    fig.show()

    return pf, current_entry, current_trend


In [ ]:
pf, current_entry, current_trend = MSTDStrategy(NMB, window=20, threshold=1.6)
print(f"The current market trend according moving standard deviation startegy for the window size of 20 is {current_trend}")
print(f"According to the moving standard deviation startegy for the window size of 20 , the backtested return is: {pf.total_return()}")
print(f"The current entry is at Date: {current_entry[0]} and value {current_entry[1]}")

The current market trend according moving standard deviation startegy for the window size of 20 is Uptrend
According to the moving standard deviation startegy for the window size of 20 , the backtested return is: 2.464048110437023
The current entry is at Date: 2011-03-20 and value 250.0


In [ ]:
import vectorbt as vbt
import pandas as pd
import plotly.graph_objects as go

def STOCHStrategy(df, k_window=14, d_window=3, overbought=80, oversold=20):
    '''
    Calculates the Stochastic Oscillator lines (%K and %D), generates entry and exit signals based on Stochastic Oscillator crossovers,
    determines the current market trend, and visualizes the market scenario with Stochastic Oscillator lines and signals.

    Parameters:
    - df (pd.DataFrame): DataFrame containing 'High', 'Low', and 'Close' prices.
    - k_window (int): Window size for %K line (default is 14).
    - d_window (int): Window size for %D line (default is 3).
    - overbought (int): Threshold for overbought condition (default is 80).
    - oversold (int): Threshold for oversold condition (default is 20).

    Returns:
    - pf (vectorbt.portfolio.base.Portfolio): Portfolio object containing trade signals.
    - current_entry (tuple): Tuple containing index and price of the most recent entry point.
    - current_trend (str): Description of the current market trend (Uptrend, Downtrend, or Sideways).
    '''
    df['High'] = df['High'].astype(str).str.replace(',', '').astype(float)
    df['Low'] = df['Low'].astype(str).str.replace(',', '').astype(float)
    df['Close'] = df['Close'].astype(str).str.replace(',', '').astype(float)
    #df['Qty'] = df['Qty'].astype(str).str.replace(',', '').astype(float)

    close = df['Close']

    # Calculate Stochastic Oscillator
    stoch = vbt.STOCH.run(df['High'], df['Low'], df['Close'], k_window=k_window, d_window=d_window)
    k = stoch.percent_k
    d = stoch.percent_d

    # Generate entry and exit signals based on Stochastic Oscillator crossovers
    entries = (k < oversold) & (k > d)
    exits = (k > overbought) & (k < d)

    # Determine the current market trend based on recent Stochastic Oscillator values
    if entries.iloc[-1]:
        current_trend = "Uptrend (Potential Reversal)"
    elif exits.iloc[-1]:
        current_trend = "Downtrend (Potential Reversal)"
    else:
        current_trend = "Sideways"

    # Create a portfolio from signals
    pf = vbt.Portfolio.from_signals(close, entries, exits)

    # Find the most recent entry point
    if entries.any():
        last_entry_index = entries[entries].index[1]
        current_entry = (last_entry_index, close[last_entry_index])
    else:
        current_entry = (None, None)

    # Plotting
    fig = go.Figure()

    # Plot the closing price
    fig.add_trace(go.Scatter(x=df.index, y=close, mode='lines', name='Close'))

    # Plot the %K line
    fig.add_trace(go.Scatter(x=k.index, y=k, mode='lines', name='%K'))

    # Plot the %D line
    fig.add_trace(go.Scatter(x=d.index, y=d, mode='lines', name='%D'))

    # Plot overbought and oversold lines
    fig.add_trace(go.Scatter(
        x=[df.index[0], df.index[-1]],
        y=[overbought, overbought],
        mode='lines',
        line=dict(dash='dash', color='red'),
        name='Overbought'
    ))

    fig.add_trace(go.Scatter(
        x=[df.index[0], df.index[-1]],
        y=[oversold, oversold],
        mode='lines',
        line=dict(dash='dash', color='green'),
        name='Oversold'
    ))

    # Plot entry signals
    fig.add_trace(go.Scatter(
        x=entries[entries].index,
        y=close[entries],
        mode='markers',
        marker=dict(symbol='triangle-up', color='green', size=10),
        name='Entry Signal'
    ))

    # Plot exit signals
    fig.add_trace(go.Scatter(
        x=exits[exits].index,
        y=close[exits],
        mode='markers',
        marker=dict(symbol='triangle-down', color='red', size=10),
        name='Exit Signal'
    ))

    # Add titles and labels
    fig.update_layout(
        title='Market Scenario with Stochastic Oscillator',
        xaxis_title='Date',
        yaxis_title='Price',
        legend_title='Legend'
    )

    # Show the plot
    fig.show()

    return pf, current_entry, current_trend


In [ ]:
pf, current_entry, current_trend = STOCHStrategy(NMB, k_window=14, d_window=3, overbought=80, oversold=20)
print(f"The current market trend according to Stochastic Oscillator startegy for k_window=14 and d_window=3 is {current_trend}")
print(f"According to the Stochastic Oscillator startegy for k_window=14 and d_window=3, the backtested return is: {pf.total_return()}")
print(f"The current entry is at Date: {current_entry[0]} and value {current_entry[1]}")

The current market trend according to Stochastic Oscillator startegy for k_window=14 and d_window=3 is Downtrend (Potential Reversal)
According to the Stochastic Oscillator startegy for k_window=14 and d_window=3, the backtested return is: -0.2067452583335333
The current entry is at Date: 2024-02-25 and value 177.6
